### Inputs: Output of EmpTitleFuzzyWuzzyAlgo_Final.ipynb; containing the employee titles resulting from Levenshtein distance
#### Script: Takes zip codes and runs it through L2 Logistic Regression Model to evaluate categorical features that have some predictive value and also joins this with the output of the EmpTitleJW.ipynb title script
### Outputs: a fully merged file containing predictive zip codes and employee titles generated from both Levenshtein and Jaro-Winkler distances.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline
import seaborn as sns
from time import time
from sklearn.preprocessing import scale
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn.metrics import make_scorer, f1_score, accuracy_score, classification_report, roc_auc_score


In [20]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [21]:
random_state=401
n_jobs = 3

In [2]:
cd Data

/Users/Shalu/Dropbox/Coursework/DS/KLC/Submission/Data


In [3]:
df_reduce = pd.read_pickle('dfc_fw_emp_1027_74.p')

In [13]:
df_reduce.head(5)

,index,loan_amnt,term,int_rate,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,initial_list_status,collections_12_mths_ex_med,mort_acc,loan_label,emp_title_flag,fico_avg,f_norm_emp_title,fw_emp_title,emp_length_cat,fw_emp_title_gt0005
0,138828,35000.0,60 months,14.49,C,C4,0-3,10,MORTGAGE,120000.0,Verified,Current,debt_consolidation,925xx,CA,14.85,0.0,14,0.0,7.0,0.0,29782.0,54.1,w,0.0,1.0,0,1,737.0,03,NoMatch,10,NoMatch
1,196305,25000.0,36 months,15.61,C,C5,0-3,5,MORTGAGE,88800.0,Verified,Charged Off,debt_consolidation,236xx,VA,19.39,0.0,12,1.0,18.0,0.0,19647.0,61.0,f,0.0,2.0,1,1,722.0,03,NoMatch,5,NoMatch
2,123573,17350.0,36 months,16.99,D,D3,04 LCDR,10,MORTGAGE,100000.0,Source Verified,Fully Paid,debt_consolidation,983xx,WA,16.61,1.0,19,0.0,12.0,0.0,29408.0,73.9,w,0.0,10.0,0,1,672.0,04 lcdr,NoMatch,10,NoMatch
3,63639,21600.0,60 months,14.99,C,C5,06 Journymen,1,MORTGAGE,61000.0,Source Verified,Fully Paid,credit_card,982xx,WA,13.18,1.0,15,0.0,8.0,0.0,27820.0,53.8,w,0.0,6.0,0,1,677.0,06 journymen,06 journymen,1,NoMatch
4,50745,8000.0,36 months,10.99,B,B3,0perator,7,MORTGAGE,100000.0,Not Verified,Current,home_improvement,945xx,CA,15.18,0.0,8,1.0,14.0,0.0,13292.0,24.8,f,0.0,1.0,0,1,702.0,0perator,0perator,7,0perator


# Lets evaluate what zip codes are important and create a new categorical feature that only takes the important zips (coef>.001)

In [14]:
df_reduce.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232487 entries, 0 to 232486
Data columns (total 33 columns):
index                         232487 non-null int64
loan_amnt                     232487 non-null float64
term                          232487 non-null object
int_rate                      232487 non-null float64
grade                         232487 non-null object
sub_grade                     232487 non-null object
emp_title                     232487 non-null object
emp_length                    232487 non-null int64
home_ownership                232487 non-null object
annual_inc                    232487 non-null float64
verification_status           232487 non-null object
loan_status                   232487 non-null object
purpose                       232487 non-null object
zip_code                      232487 non-null object
addr_state                    232487 non-null object
dti                           232487 non-null float64
delinq_2yrs                   232487 no

In [15]:
random_state = 401

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import scale
from sklearn.metrics import make_scorer, f1_score, accuracy_score, classification_report, roc_auc_score
from sklearn.model_selection import train_test_split

def my_scorer(clf, X, y):
    y_pred_prob = clf.predict_proba(X)[:,1]
    return roc_auc_score(y, y_pred_prob)

sss = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=random_state)
scoring_function = my_scorer

In [16]:
logreg = LogisticRegression(penalty='l2', C=.00001, random_state=random_state)

In [17]:
features = ['zip_code']

target = ['loan_label']

X=df_reduce[features]
print X.columns
X = pd.get_dummies(X)
col = X.columns
print X.shape
#X = scale(X)
y=df_reduce[target].values.ravel()

X_train, X_test, y_train, y_test = train_test_split(X, y,  stratify=y, test_size=.25, random_state=random_state)

Index([u'zip_code'], dtype='object')
(232487, 865)


In [15]:
C_range = 1.0/np.power(10,np.arange(0,8,1))
print C_range
parameters=dict(C=C_range)
grid_log = GridSearchCV(estimator=logreg, param_grid=parameters, cv=sss, n_jobs=n_jobs, scoring=my_scorer, refit=True)

[  1.00000000e+00   1.00000000e-01   1.00000000e-02   1.00000000e-03
   1.00000000e-04   1.00000000e-05   1.00000000e-06   1.00000000e-07]


In [19]:
start = time()
grid_log.fit(X_train,y_train)
end = time()
print "{:.4f} seconds".format(end - start)
y_pred_prob = logreg.predict_proba(X_test)[:,1]
print roc_auc_score(y_test, y_pred_prob)

61.4620 seconds
0.506659308846


In [20]:
t = pd.DataFrame(zip(col, logreg.coef_[0]), columns=['features','coef'])
t['coef']=abs(t.coef)
t.sort_values(by='coef', ascending=False).head(15)

,features,coef
646,zip_code_750xx,0.005365
530,zip_code_606xx,0.004904
815,zip_code_945xx,0.004631
90,zip_code_112xx,0.004335
271,zip_code_300xx,0.004263
78,zip_code_100xx,0.004087
51,zip_code_070xx,0.003871
666,zip_code_770xx,0.003770
302,zip_code_331xx,0.003458
770,zip_code_900xx,0.003354


In [37]:
print len(t[t.coef>0.001]) #want to reduce these zip codes by most important 
#take coefficients greater than .001
emp001 = t[t.coef>0.001]

131


In [38]:
print len('zip_code_')
#remove dummy column string concatenated and 
emp001['zip_red'] = emp001.features.apply(lambda x: x[9:])

9


/Users/Shalu/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [55]:
#take top coefficient zip codes and set lower to 0
def map_top_matches(x):
        if any(x == (emp001.zip_red)):
            return x
        else:
            return 0

In [56]:
start=time()
df_reduce['zip_gt001'] = df_reduce.zip_code.apply(lambda x: map_top_matches(x))
end = time()
print "{:.4f} seconds".format((end - start))

28.2132 seconds


In [19]:
#Reduced the ZipCodes from 865 to 132
print len(np.unique(df_reduce.zip_code))
print len(np.unique(df_reduce.zip_gt001))

865
132


In [18]:
#df_reduce.to_pickle('deltempZipFuzzyMerge.p')
#df_reduce = pd.read_pickle('deltempZipFuzzyMerge.p')
df_reduce = pd.read_pickle('dfc_zip_fuzzy_final.p')